In [ ]:
from cvzone.HandTrackingModule import HandDetector
import cv2
import os
import numpy as np

# Parameters
width, height = 1280, 720
gestureThreshold = 700
folderPath = "Presentation"
brushesFolderPath = "brushes"

# Camera Setup
cap = cv2.VideoCapture(0)
cap.set(3, width)
cap.set(4, height)

# Hand Detector
detectorHand = HandDetector(detectionCon=0.8, maxHands=1)

# Variables
imgList = []
delay = 10
buttonPressed = False
counter = 0
drawMode = False
imgNumber = 0
delayCounter = 0
annotations = [[]]
annotationNumber = -1
annotationStart = False
hs, ws = int(120 * 1), int(210 * 1)  # width and height of small image

# Get list of presentation images
pathImages = sorted(os.listdir(folderPath), key=len)
print(pathImages)

overlayList = []
for imPath in pathImages:
    image = cv2.imread(os.path.join(folderPath, imPath))
    overlayList.append(image)

header = overlayList[0]
drawColor = (255, 0, 255)

xp, yp = 0, 0

# Colors for selection
colorRectangles = []
colorList = [(255, 0, 255, 100), (0, 0, 255, 100), (0, 255, 0, 100), (255, 255, 255, 100)]

pinkImage = cv2.imread(os.path.join(brushesFolderPath, "pink.png"))
redImage = cv2.imread(os.path.join(brushesFolderPath, "red.png"))
greenImage = cv2.imread(os.path.join(brushesFolderPath, "green.png"))
eraserImage = cv2.imread(os.path.join(brushesFolderPath, "eraser.png"))

colorImages = {
    "Pink": pinkImage,
    "Red": redImage,
    "Green": greenImage,
    "Eraser": eraserImage
}

annotationsCanvas = np.zeros((height, width, 3), np.uint8)

selectedImage = ""

while True:
    # Get image frame
    success, img = cap.read()
    img = cv2.flip(img, 1)
    pathFullImage = os.path.join(folderPath, pathImages[imgNumber])
    imgCurrent = cv2.imread(pathFullImage)

    for i, (name, colorImg) in enumerate(colorImages.items()):
        imgCurrent[0:100, i * 100:(i + 1) * 100] = colorImg
        x1 = 0 + i * 100
        y1 = 0
        x2 = 100 + i * 100
        y2 = 100
        colorRectangles.append((x1, y1, x2, y2))
    
    # Find the hand and its landmarks
    hands, img = detectorHand.findHands(img)  # with draw
    # Draw Gesture Threshold line
    cv2.line(img, (0, gestureThreshold), (width, gestureThreshold), (0, 255, 0), 10)

    if hands and buttonPressed is False:  # If hand is detected
        hand = hands[0]
        cx, cy = hand["center"]
        lmList = hand["lmList"]  # List of 21 Landmark points
        fingers = detectorHand.fingersUp(hand)  # List of which fingers are up

        # Constrain values for easier drawing
        xVal = int(np.interp(lmList[8][0], [width // 2, width], [0, width]))
        yVal = int(np.interp(lmList[8][1], [150, height-150], [0, height]))
        indexFinger = xVal, yVal

        # Check if hand is in the color selection area
        for i, rect in enumerate(colorRectangles):
            if rect[0] < xVal < rect[2] and rect[1] < yVal < rect[3]:
                selectedImage = list(colorImages.keys())[i]
                break

        # Update drawColor based on selectedImage
        if selectedImage == "Pink":
            drawColor = (255, 0, 255, 255)
        elif selectedImage == "Red":
            drawColor = (0, 0, 255)
        elif selectedImage == "Green":
            drawColor = (0, 255, 0)
        elif selectedImage == "Eraser":
            drawColor = (0, 0, 0, 0)

        if cy <= gestureThreshold:  # If hand is at the height of the face
            cv2.rectangle(img, (lmList[8][1], lmList[8][2] - 25), (lmList[12][1], lmList[12][2] + 25), drawColor, cv2.FILLED)
            if fingers == [1, 0, 0, 0, 0]:
                print("Left")
                buttonPressed = True
                
                if imgNumber > 0:
                    imgNumber -= 1
                    annotations = [[]]
                    annotationNumber = -1
                    annotationStart = False
                    annotationsCanvas = np.zeros((height, width, 3), np.uint8)
            if fingers == [0, 0, 0, 0, 1]:
                print("Right")
                buttonPressed = True
                if imgNumber < len(pathImages) - 1:
                    imgNumber += 1
                    annotations = [[]]
                    annotationNumber = -1
                    annotationStart = False
                    annotationsCanvas = np.zeros((height, width, 3), np.uint8)
                    
            if fingers == [0, 1, 1, 0, 0]:
                cv2.circle(imgCurrent, indexFinger, 12, drawColor, cv2.FILLED)

            # Drawing Mode
            if fingers == [0, 1, 0, 0, 0]:
                if yVal > 100 or xVal > 500:
                    if annotationStart is False:
                        annotationStart = True
                        annotationNumber += 1
                        annotations.append([])
                    annotations[annotationNumber].append((indexFinger, drawColor))  # Store point and color
                cv2.circle(imgCurrent, indexFinger, 12, drawColor, cv2.FILLED)
            else:
                annotationStart = False

        if fingers == [0, 1, 1, 1, 0]:
            if annotations:
                annotations.pop(-1)
                annotationNumber -= 1
                buttonPressed = True

    else:
        annotationStart = False
    if buttonPressed:
        counter += 1
        if counter > delay:
            counter = 0
            buttonPressed = False
            
    for annotation in annotations:
        for i in range(1, len(annotation)):
            # Draw the line with the selected color
            cv2.line(imgCurrent, annotation[i - 1][0], annotation[i][0], annotation[i][1], 12)
    
    #annotationsCanvas = cv2.resize(annotationsCanvas, (imgCurrent.shape[1], imgCurrent.shape[0]))
    #imgCurrent = cv2.addWeighted(imgCurrent, 1, annotationsCanvas, 1, 0)
    
    imgSmall = cv2.resize(img, (ws, hs))
    h, w, _ = imgCurrent.shape
    imgCurrent[0:hs, w - ws: w] = imgSmall

    cv2.imshow("Slides", imgCurrent)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg']
Right
Right
Left
Left
Left
Left
Left
Left
Left
Left
Left
Right
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Right
Left
Right
Right
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Right
Right
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
